In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, concatenate, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
import tifffile as tiff

In [ ]:
# Data directories
data_dir = '/content/drive/Shareddrives/DeepLearning_2024/FinalProject/Data'

train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Load and preprocess the data
def load_data_from_directory(base_dir, img_size=(224, 224)):
    categories = ['polyps', 'non_polyps']
    images = []
    labels = []

    for category in categories:
        path = os.path.join(base_dir, category)
        class_num = categories.index(category)

        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img = load_img(img_path, target_size=img_size)
            img = img_to_array(img) / 255.0
            images.append(img)
            labels.append(class_num)

    images = np.array(images)
    labels = np.array(labels)

    # Standardize images
    mean = np.mean(images)
    std = np.std(images)
    images = (images - mean) / std

    return images, labels

# Load training and test data
train_images, train_labels = load_data_from_directory(train_dir)
test_images, test_labels = load_data_from_directory(test_dir)

# Split training data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

In [ ]:
# VGG16 model for transfer learning
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding custom layers for classification
inputs = Input(shape=(224, 224, 3))
base_model = vgg16(inputs)
x = GlobalAveragePooling2D()(base_model)
x = Dense(256, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

vgg16_model = Model(inputs, outputs)

# Freeze all layers except the last two dense layers
for layer in vgg16_model.layers[:-4]:
    layer.trainable = False

# Compile the model
vgg16_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# Train the model with data augmentation
vgg16_history = vgg16_model.fit(train_datagen.flow(train_images, train_labels, batch_size=8),
                                epochs=50,
                                validation_data=(val_images, val_labels))

# Evaluate the model
loss, accuracy = vgg16_model.evaluate(test_images, test_labels)
print(f'Loss: {loss}, Accuracy: {accuracy}')


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/50
88/91 [============================>.] - ETA: 46s - loss: 0.5901 - accuracy: 0.6662 

KeyboardInterrupt: 

In [ ]:
plot_history(vgg16_history)

In [ ]:

# Function to resize images
def resize_images(images, size=(160, 160)):
    resized_images = np.zeros((images.shape[0], size[0], size[1], 3))
    for i in range(images.shape[0]):
        resized_img = array_to_img(images[i]).resize(size)
        resized_images[i] = img_to_array(resized_img)
    return resized_images

# Resize train, validation, and test images
train_images_resized = resize_images(train_images, size=(160, 160))
val_images_resized = resize_images(val_images, size=(160, 160))
test_images_resized = resize_images(test_images, size=(160, 160))

# InceptionV3 model for transfer learning
inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(160, 160, 3))

# Adding custom layers for classification
inputs = Input(shape=(160, 160, 3))
base_model = inceptionv3(inputs)
x = GlobalAveragePooling2D()(base_model)
x = Dense(256, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

inceptionv3_model = Model(inputs, outputs)

# Freeze all layers except the last two dense layers
for layer in inceptionv3_model.layers[:-4]:
    layer.trainable = False

# Compile the model
inceptionv3_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# Train the model with data augmentation
inceptionv3_history = inceptionv3_model.fit(
    train_datagen.flow(train_images_resized, train_labels, batch_size=8),
    epochs=50,
    validation_data=(val_images_resized, val_labels))

# Evaluate the model
loss, accuracy = inceptionv3_model.evaluate(test_images_resized, test_labels)
print(f'Loss: {loss}, Accuracy: {accuracy}')


In [ ]:
plot_history(inceptionv3_history)


In [ ]:

# ResNet50 model for transfer learning
resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding custom layers for classification
inputs = Input(shape=(224, 224, 3))
base_model = resnet50(inputs)
x = GlobalAveragePooling2D()(base_model)
x = Dense(256, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x)

resnet50_model = Model(inputs, outputs)

# Freeze all layers except the last two dense layers
for layer in resnet50_model.layers[:-4]:
    layer.trainable = False

# Compile the model
resnet50_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# Train the model with data augmentation
resnet50_history = resnet50_model.fit(train_datagen.flow(train_images, train_labels, batch_size=8),
                                      epochs=50,
                                      validation_data=(val_images, val_labels))

# Evaluate the model
loss, accuracy = resnet50_model.evaluate(test_images, test_labels)
print(f'Loss: {loss}, Accuracy: {accuracy}')


In [ ]:
plot_history(resnet50_history)


In [ ]:
# Load the pre-trained VGG16 model
#model = vgg16_model
#model = resnet50_model
model = inceptionv3_model

image_path = data_dir + '/144.tif'

# Load the image using tifffile
try:
    img = tiff.imread(image_path)
    print(f"Image loaded successfully. Shape: {img.shape}")
except Exception as e:
    print(f"Failed to load image: {e}")
    img = None


# If the image is grayscale, convert it to BGR
if len(img.shape) == 2 or img.shape[2] == 1:
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    print("Converted grayscale image to BGR.")

# Resize the image to ensure it fits the sliding window process
if img.shape[0] < 224 or img.shape[1] < 224:
    img = cv2.resize(img, (224, 224))
    print(f"Resized image to: {img.shape}")

# Define sliding window parameters
WINDOW_SIZE = 150
step = 10

# Initialize variables to store the best prediction
max_pred = 0.0
max_box = []

print('--> Searching for a colonoscopy polyp ...')

# Loop through the image using sliding window
for top in range(0, img.shape[0] - WINDOW_SIZE + 1, step):
    for left in range(0, img.shape[1] - WINDOW_SIZE + 1, step):
        # Compute the coordinates of the bounding box
        box = (top, left, top + WINDOW_SIZE, left + WINDOW_SIZE)

        # Crop the original image
        cropped_img = img[box[0]:box[2], box[1]:box[3], :]
        # Resize the cropped image to match the input size expected by the model
        cropped_img_resized = cv2.resize(cropped_img, (160, 160))

        # Normalize the resized cropped image
        # cropped_img_resized = cropped_img * 1.0 / 255
        cropped_img_resized = cropped_img_resized * 1.0 / 255
        # Reshape for prediction
        cropped_img_resized = cropped_img_resized.reshape((1, cropped_img_resized.shape[0], cropped_img_resized.shape[1], cropped_img_resized.shape[2]))

        # Make prediction for the resized cropped image
        preds = model.predict(cropped_img_resized)

        if preds[0][0] > max_pred:
            max_pred = preds[0][0]
            max_box = box

print('Done!')
print('Best prediction:', max_box, max_pred)

# Visualize the best prediction box on the original image
plt.figure()
plt.imshow(img)
plt.text(1, -5, 'Best probability: ' + str(max_pred), fontsize=10)
currentAxis = plt.gca()
if max_box:  # Check if max_box is not empty
    currentAxis.add_patch(Rectangle((max_box[1], max_box[0]), WINDOW_SIZE, WINDOW_SIZE, linewidth=1, edgecolor='r', facecolor='none'))
plt.show()
